In [1]:
import albumentations
import albumentations.pytorch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from sklearn.metrics import f1_score
from efficientnet_pytorch import EfficientNet

tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

cuda:0 is using!


## WandB INIT, Config

In [3]:
import wandb

config={
    "user" : "JH",
    "lr": 3e-4,
    "architecture": "efficientnet-b5",
    "dataset": "facecrop",
    "augmentation" : "cutmix",
    "loss" : "F1loss",
    "gamma" : 0.9,
    "scheduler" : "cosine",
    "batch_size" : 16,
    "epochs" : 30,
    "beta" : 1.0,
    "memo" : "MSD3"
}
config['model'] = '_'.join([config['user'], config['architecture'], config["memo"], config['dataset'], config['augmentation'], config['loss'], str(config['batch_size']), str(config['lr']), config["scheduler"]])
wandb.init(entity="minibatch28", project="MaskClassification", name=config['model'], config=config)

wandb: Currently logged in as: minibatch28 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



## Dataset

In [4]:
class MyDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']  # transforms를 사용하시는 분은 X = self.transform(X)
        return X, y

In [5]:
class TestDataset(Dataset):
    def __init__(self, path, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X = self.X[idx]
        X = self.transform(image=X)['image']  # transforms를 사용하시는 분은 X = self.transform(X)
        return X

In [6]:
# Augmentation은 각자 방식대로

train_transform = albumentations.Compose(
  [
      albumentations.Resize(456,456),
#       albumentations.RandomRotation(15),
      albumentations.HorizontalFlip(p=0.5),
      albumentations.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.6),
      albumentations.RandomBrightnessContrast(brightness_limit=0.1, p=0.6),
      albumentations.GaussNoise(p=0.5),
      albumentations.MotionBlur(p=0.5),
      albumentations.OpticalDistortion(p=0.5),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2(),
      #       이미지 원본 사이즈는 384, 512   
  ]
)

test_transform = albumentations.Compose(
  [
      albumentations.Resize(456,456),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2()
      #       이미지 원본 사이즈는 384, 512   
  ]
)

## DataLoader

In [7]:
# DF 읽어오기
train_df = pd.read_csv('/opt/ml/input/face_train_df.csv')
valid_df = pd.read_csv('/opt/ml/input/face_valid_df.csv')

# Dataset 객체 생성
dataset_train = MyDataset(path=train_df['full_path'].values,
                          label=train_df['label'].values,
                          transform=train_transform)

dataset_valid = MyDataset(path=valid_df['full_path'].values,
                          label=valid_df['label'].values,
                          transform=test_transform)

# Loader 올리기
train_dataloader = DataLoader(dataset_train, batch_size=config['batch_size'], shuffle=True, num_workers=3, pin_memory=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=config['batch_size'], shuffle=False, num_workers=3, pin_memory=True)

100%|██████████| 3780/3780 [00:05<00:00, 719.01it/s]


In [8]:
from efficientnet_pytorch import EfficientNet
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.backbone = EfficientNet.from_pretrained(config['architecture'], num_classes=1024)
        self.fc = nn.Linear(1024, 18)
        self.ms1 = nn.Sequential(
            nn.Dropout(0.5),
            self.fc
        )
        self.ms2 = nn.Sequential(
            nn.Dropout(0.5),
            self.fc
        )
        self.ms3 = nn.Sequential(
            nn.Dropout(0.5),
            self.fc
        )
        self.num_samples = 3
        self.best_f1 = 0

    def forward(self, x):
        x = self.backbone(x)
        output1 = self.ms1(x)
        output2 = self.ms2(x)
        output3 = self.ms3(x)
        
        x = (output1 + output2 + output3) / self.num_samples
        return x
        
model = Net().to(device)

Loaded pretrained weights for efficientnet-b5


## Loss, Optimizer, Scheduler

### Weightedloss

In [9]:
import torch.optim as optim 

class_distribution = torch.tensor([2745, 2050, 415, 3660, 4085, 545, 
                      555, 314, 179, 726, 669, 257, 
                      555, 314, 179, 726, 669, 257])
normedWeights = [1 - (x / sum(class_distribution)) for x in class_distribution]
loss_distribution = class_distribution / class_distribution.sum()
loss_distribution = 1.0 / loss_distribution
loss_distribution = (loss_distribution / loss_distribution.sum()*10).to(device)
print(loss_distribution)

tensor([0.0904, 0.1211, 0.5981, 0.0678, 0.0608, 0.4554, 0.4472, 0.7905, 1.3867,
        0.3419, 0.3710, 0.9658, 0.4472, 0.7905, 1.3867, 0.3419, 0.3710, 0.9658],
       device='cuda:0')


In [10]:
# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1Loss(nn.Module):
    def __init__(self, classes=18, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

criterion = F1Loss()
# criterion = nn.CrossEntropyLoss(weight=loss_distribution)
# criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=config['gamma'], verbose=True)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=6, T_mult=1, eta_min=5e-6, last_epoch=-1, verbose=True)

Epoch     0: adjusting learning rate of group 0 to 3.0000e-04.


## Validation Function

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def check():
    model.eval()
    val_loss = 0.0
    counter = 0
    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        for x, y in valid_dataloader:
            counter += 1
            y_true += y.tolist()
            
            x = x.to(device=device)
            y = y.to(device=device)
    
            scores = model(x)
            _, predictions = scores.max(1)
            y_predicted += predictions.tolist()
            
            
            loss = criterion(scores, y)
            
            val_loss += loss.item()
#     print(y_true, y_predicted)
    cm = confusion_matrix(y_true, y_predicted)
    F1 = []
    epsilon = 1e-7
    for c in range(18):
        precision = cm[c][c] / (np.sum(cm, axis=0)[c] + epsilon)
        recall = cm[c][c] / (np.sum(cm, axis=1)[c] + epsilon)
        f = 2 * precision * recall / (precision + recall)
        if np.isnan(f):
            f = 0
        F1.append(f)
    macro_F1 = np.mean(F1)
    
    print("< VALIDATION >")
    print("*"*73)
    print("Validation Loss :", val_loss/counter)
    print("-"*73)
    print("Total Accuracy")
    print(accuracy_score(y_true, y_predicted) * 100, "%")
    print("-"*73)
    print("Confusion Matrix")
    for row in cm:
        for c in row:
            print(str(c).ljust(4), end='')
        print()
    print("-"*73)
    print("Validation F1 score :" , macro_F1)
    for c, f in enumerate(F1):
        print("Class", c, ":", f)
        
    if model.best_f1 < macro_F1:
        model.best_f1 = macro_F1
#     if model.best_valid > val_loss/counter:
#         model.best_valid = val_loss/counter
        torch.save(model.state_dict(), '/opt/ml/weights/{}/{:.4f}.pt'.format(config['model'], model.best_f1))
    print("model saved!")
    print("*"*73)
    print()
    wandb.log({
        "Validation Loss" : val_loss/counter, 
        "Validation Total Accuracy" :accuracy_score(y_true, y_predicted) * 100, 
        "Validation F1" : macro_F1,
        "Learning Rate" : scheduler.get_last_lr()[0]*10000
    })
        
    model.train()

## Cutmix

In [12]:
def rand_bbox(size, lam): # size : [Batch_size, Channel, Width, Height]
    W = size[2] 
    H = size[3] 
    cut_rat = np.sqrt(1. - lam)  # 패치 크기 비율
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)  

   	# 패치의 중앙 좌표 값 cx, cy
    cx = np.random.randint(W)
    cy = np.random.randint(H)
		
    # 패치 모서리 좌표 값 
    bbx1 = 0
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = W
    bby2 = np.clip(cy + cut_h // 2, 0, H)
   
    return bbx1, bby1, bbx2, bby2

## Model Training

In [ ]:
folder = '/opt/ml/weights/{}'.format(config['model'])
if not os.path.exists(folder):
    os.mkdir(folder)

for epoch in range(config['epochs']):
    print("Epoch :", epoch + 1)
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    total = 0
    train_running_loss = 0.0
    train_running_correct = 0
    
    total_it = int(len(dataset_train)/train_dataloader.batch_size)
    prog_bar = tqdm(enumerate(train_dataloader), total=total_it)
    for i, (images, labels) in prog_bar:
        
        counter += 1
        total += labels.size(0)
        
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        
        if "cutmix" in config['augmentation']:
            if config['beta'] > 0 and np.random.random() > 0.5: # cutmix가 실행될 경우     
                    lam = np.random.beta(config['beta'], config['beta'])
                    rand_index = torch.randperm(images.size()[0]).to(device)
                    target_a = labels # 원본 이미지 label
                    target_b = labels[rand_index] # 패치 이미지 label       
                    bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(), lam)
                    images[:, :, bbx1:bbx2, bby1:bby2] = images[rand_index, :, bbx1:bbx2, bby1:bby2]
                    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.size()[-1] * images.size()[-2]))
                    outputs = model(images)
                    loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam) # 패치 이미지와 원본 이미지의 비율에 맞게 loss를 계산을 해주는 부분

            else: # cutmix가 실행되지 않았을 경우
                outputs = model(images) 
                loss = criterion(outputs, labels)
        
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            counter += 1
            
           
        _, preds= torch.max(outputs, 1) 
#         _, preds = torch.max(outputs.data, 1)
        
        
        
        loss.backward()

        optimizer.step() 
        train_running_loss += loss.item()
        train_running_correct += (preds == labels).sum().item()
        
        
        del images
        del labels
        torch.cuda.empty_cache()
        
        
        
        if i == total_it//2 or i == total_it-1:
            train_loss = train_running_loss / counter
            train_accuracy = 100. * train_running_correct / total
            
            print("Train Loss :", train_loss)
            print("Train Accuracy :", train_accuracy)
            wandb.log({"Train Loss" : train_loss,
                      "Train Accuracy" : train_accuracy})
            try:
                check()
            except:
                pass
    scheduler.step()
print("Finish")

Epoch : 1


 50%|████▉     | 472/945 [07:10<07:09,  1.10it/s]

Train Loss : 0.7824988973821192
Train Accuracy : 45.62632135306554


<ipython-input-11-f0f531eb0249>:34: RuntimeWarning: invalid value encountered in double_scalars
  f = 2 * precision * recall / (precision + recall)


< VALIDATION >
*************************************************************************
Validation Loss : 0.8598970946883351
-------------------------------------------------------------------------
Total Accuracy
57.08994708994709 %
-------------------------------------------------------------------------
Confusion Matrix
365 34  12  2   104 0   14  8   0   0   9   0   1   0   0   0   1   0   
5   65  30  0   36  4   0   68  0   0   42  0   0   62  0   0   3   0   
0   7   50  0   3   2   0   41  0   0   5   0   0   71  0   0   1   0   
14  1   0   488 203 1   0   0   0   8   11  0   0   2   0   0   2   0   
0   0   0   3   510 5   0   0   0   0   112 0   0   5   0   0   35  0   
0   0   0   0   164 9   0   0   0   0   46  0   0   1   0   0   35  0   
3   0   0   0   0   0   76  3   0   0   28  0   0   0   0   0   0   0   
0   0   0   0   0   0   1   22  0   0   40  0   0   0   0   0   0   0   
0   0   0   0   0   0   0   17  0   0   18  0   0   1   0   0   0   0   
0   0   0   0   0

...

<ipython-input-11-f0f531eb0249>:34: RuntimeWarning: invalid value encountered in double_scalars
  f = 2 * precision * recall / (precision + recall)
100%|██████████| 945/945 [15:25<00:00, 11.17s/it]

< VALIDATION >
*************************************************************************
Validation Loss : 0.7965419282390096
-------------------------------------------------------------------------
Total Accuracy
77.3015873015873 %
-------------------------------------------------------------------------
Confusion Matrix
523 2   5   10  6   0   4   0   0   0   0   0   0   0   0   0   0   0   
16  169 113 0   14  0   0   3   0   0   0   0   0   0   0   0   0   0   
0   44  134 0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   
13  0   0   683 31  1   0   0   0   2   0   0   0   0   0   0   0   0   
9   14  22  15  492 110 0   0   0   0   8   0   0   0   0   0   0   0   
0   3   2   0   155 93  0   0   0   0   2   0   0   0   0   0   0   0   
1   0   0   0   0   0   100 2   1   4   2   0   0   0   0   0   0   0   
0   1   0   0   0   0   2   37  18  0   5   0   0   0   0   0   0   0   
0   0   1   0   0   0   0   8   26  0   1   0   0   0   0   0   0   0   
0   0   0   4   0 

100%|██████████| 945/945 [15:25<00:00,  1.02it/s]

Epoch     9: adjusting learning rate of group 0 to 1.5250e-04.
Epoch : 10



 35%|███▌      | 334/945 [05:04<09:18,  1.09it/s]

## Load Best Model

In [ ]:
model.load_state_dict(torch.load('/opt/ml/weights/{}/{:.4f}.pt'.format(config['model'], model.best_f1))), model.best_f1

# Q = "0.7737"
# model.load_state_dict(torch.load('/opt/ml/weights/{}/{}.pt'.format(config['model'], Q)))
# model.best_f1 = Q

## Testing

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'new_images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, transform=test_transform)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=32,
    num_workers=3
)

model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []

prog_bar = tqdm(enumerate(loader), total=int(len(dataset)/loader.batch_size))
for i, images in prog_bar:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, '{}_{}.csv'.format(config['model'], model.best_f1)), index=False)
print('test inference is done!', config['model'])

In [ ]:

# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'new_images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

idx = 200
image = cv2.imread(image_paths[idx])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

image = test_transform(image=image)['image'].to(device)
image = image.unsqueeze(0)
pred = model(image)
label = np.argmax(pred.detach().cpu().numpy())

masklabel = {0: "Mask", 1: "Incorrect", 2: "Normal"}
genderlabel = {0: "Male", 1: "Female"}
agelabel = {0: "~ 30", 1: "30 ~ 60", 2: "60 ~"}

feature_to_label = {}
features = [(m, g, a) for m in ['Mask', 'Incorrect', 'Normal'] for g in ['Male', 'Female'] for a in ["~ 30", "30 ~ 60", "60 ~"]]
for i, (m, g, a) in enumerate(features):
    feature_to_label[(m, g, a)] = i

label_to_feature = { feature_to_label[k]:k for k in feature_to_label}
m, g, a = label_to_feature[label]
print(m)
print(g)
print(a)
